In [ ]:
#Moving Average Approach:
import pandas as pd

# Create a DataFrame with quarterly data
data = {
    'Quarter': ['Q1', 'Q2', 'Q3', 'Q4', 'Q1', 'Q2', 'Q3', 'Q4'],
    'Value': [100, 110, 120, 130, 140, 150, 160, 170]
}

df = pd.DataFrame(data)

# Calculate average value for each quarter
quarterly_avg = df.groupby('Quarter')['Value'].mean()

# Calculate the overall average
overall_avg = df['Value'].mean()

# Calculate the seasonality index
seasonality_index = quarterly_avg / overall_avg

print(seasonality_index)


In [ ]:
#Decomposition approach
import pandas as pd
from statsmodels.tsa.seasonal import seasonal_decompose

# Create a DataFrame with quarterly data
data = {
    'Quarter': ['Q1', 'Q2', 'Q3', 'Q4', 'Q1', 'Q2', 'Q3', 'Q4'],
    'Value': [100, 110, 120, 130, 140, 150, 160, 170]
}

df = pd.DataFrame(data)

# Convert the DataFrame to a time series
df['Quarter'] = pd.to_datetime(df['Quarter'], format='Q%m').dt.to_period('Q')
df.set_index('Quarter', inplace=True)

# Perform seasonal decomposition
decomposition = seasonal_decompose(df['Value'], model='additive',period=4)

# Extract the seasonal component
seasonal_component = decomposition.seasonal

# Calculate the average seasonal component for each quarter
quarterly_avg_seasonality = seasonal_component.groupby(seasonal_component.index.quarter).mean()

# Calculate the overall average
overall_avg = df['Value'].mean()

# Calculate the seasonality index
seasonality_index = quarterly_avg_seasonality / overall_avg

print(seasonality_index)


In [ ]:
import pandas as pd
from statsmodels.tsa.statespace.sarimax import SARIMAX

# Create a DataFrame with quarterly data
data = {
    'Quarter': ['Q1', 'Q2', 'Q3', 'Q4', 'Q1', 'Q2', 'Q3', 'Q4'],
    'Value': [100, 110, 120, 130, 140, 150, 160, 170]
}

df = pd.DataFrame(data)

# Convert the DataFrame to a time series
df['Quarter'] = pd.to_datetime(df['Quarter'], format='Q%m').dt.to_period('Q')
df.set_index('Quarter', inplace=True)

# Fit SARIMA model
model = SARIMAX(df['Value'], order=(0, 0, 0), seasonal_order=(1, 0, 0, 4))
results = model.fit()

# Extract the seasonal component from the fitted values
seasonal_component = results.fittedvalues.diff(periods=4)

# Calculate the average seasonal component for each quarter
quarterly_avg_seasonality = seasonal_component.groupby(seasonal_component.index.quarter).mean()

# Calculate the overall average
overall_avg = df['Value'].mean()

# Calculate the seasonality index
seasonality_index = quarterly_avg_seasonality / overall_avg

print(seasonality_index)


In [ ]:
import pandas as pd
import statsmodels.api as sm

# Create a DataFrame with quarterly data
data = {
    'Quarter': ['2021-03-31', '2021-06-30', '2021-09-30', '2021-12-31', '2022-03-31', '2022-06-30', '2022-09-30', '2022-12-31'],
    'Value': [100, 110, 120, 130, 140, 150, 160, 170]
}

df = pd.DataFrame(data)

# Convert the quarters to datetime objects
df['Quarter'] = pd.to_datetime(df['Quarter'], format='%Y-%m-%d')

# Convert the quarters to numeric values
df['Quarter_Num'] = df['Quarter'].dt.quarter.astype(int)

# Create dummy variables for each quarter
quarter_dummies = pd.get_dummies(df['Quarter_Num'])

# Add an intercept column
quarter_dummies['Intercept'] = 1

# Fit a regression model
model = sm.OLS(df['Value'], quarter_dummies)
results = model.fit()

# Extract the coefficients from the regression results
coefficients = results.params

# Calculate the seasonality index
seasonality_index = coefficients[1:-1] / coefficients[-1]

# Print the seasonality index
print(seasonality_index)


In [ ]:
import pandas as pd
import numpy as np

# Create a DataFrame with quarterly data
data = {
    'Quarter': ['Q1', 'Q2', 'Q3', 'Q4', 'Q1', 'Q2', 'Q3', 'Q4'],
    'Value': [100, 110, 120, 130, 140, 150, 160, 170]
}

df = pd.DataFrame(data)

# Convert the quarters to numeric values
df['Quarter_Num'] = pd.to_datetime(df['Quarter'], format='Q%m').dt.quarter

# Calculate the average value for each quarter
quarterly_avg = df.groupby('Quarter_Num')['Value'].mean()

# Calculate the overall average
overall_avg = df['Value'].mean()

# Calculate the detrended values
detrended_values = df['Value'] / quarterly_avg[df['Quarter_Num']].values * overall_avg

# Calculate the seasonality index
seasonality_index = detrended_values.groupby(df['Quarter_Num']).mean()

print(seasonality_index)


In [ ]:
import pandas as pd
from statsmodels.tsa.seasonal import seasonal_decompose

# Create a DataFrame with quarterly data
data = {
    'Quarter': ['Q1', 'Q2', 'Q3', 'Q4', 'Q1', 'Q2', 'Q3', 'Q4'],
    'Value': [100, 110, 120, 130, 140, 150, 160, 170]
}

df = pd.DataFrame(data)

# Convert the quarters to numeric values
df['Quarter_Num'] = pd.to_datetime(df['Quarter'], format='Q%m').dt.quarter

# Convert the DataFrame to a time series
df['Quarter_Num'] = pd.to_datetime(df['Quarter_Num'], format='%m').dt.to_period('Q')
df.set_index('Quarter_Num', inplace=True)

# Perform seasonal decomposition
decomposition = seasonal_decompose(df['Value'], model='additive',period=4)

# Extract the seasonal component
seasonal_component = decomposition.seasonal

# Calculate the average seasonal component for each quarter
quarterly_avg_seasonality = seasonal_component.groupby(seasonal_component.index).mean()

# Calculate the overall average
overall_avg = df['Value'].mean()

# Calculate the seasonality index
seasonality_index = quarterly_avg_seasonality / overall_avg

print(seasonality_index)


In [ ]:
import pandas as pd
import numpy as np
from scipy.fft import fft, ifft

# Create a DataFrame with quarterly data
data = {
    'Quarter': ['Q1', 'Q2', 'Q3', 'Q4', 'Q1', 'Q2', 'Q3', 'Q4'],
    'Value': [100, 110, 120, 130, 140, 150, 160, 170]
}

df = pd.DataFrame(data)

# Convert the quarters to numeric values
df['Quarter_Num'] = pd.to_datetime(df['Quarter'], format='Q%m').dt.quarter

# Create a periodic signal based on the quarterly values
signal = np.tile(df['Value'], int(np.ceil(df.shape[0] / 4)))[:df.shape[0]]

# Apply Fourier transformation
fourier = fft(signal)

# Identify the indices corresponding to the quarterly frequencies
quarter_freq_indices = np.arange(1, signal.shape[0], 4)

# Calculate the magnitudes of the quarterly frequencies
magnitudes = np.abs(fourier[quarter_freq_indices]) / df.shape[0]

# Calculate the average magnitude for each quarter
quarterly_avg_magnitudes = magnitudes.mean()

# Calculate the overall average
overall_avg = df['Value'].mean()

# Calculate the seasonality index
seasonality_index = quarterly_avg_magnitudes / overall_avg

print(seasonality_index)


In [ ]:
"""
This linear regression approach estimates the seasonality index by modeling the relationship 
between the quarters (represented as one-hot encoded variables) and the values. It captures 
the relative contribution of each quarter to the overall pattern.
"""

import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression

# Create a DataFrame with quarterly data
data = {
    'Quarter': ['Q1', 'Q2', 'Q3', 'Q4', 'Q1', 'Q2', 'Q3', 'Q4'],
    'Value': [100, 110, 120, 130, 140, 150, 160, 170]
}

df = pd.DataFrame(data)

# Convert the quarters to numeric values
df['Quarter_Num'] = pd.to_datetime(df['Quarter'], format='Q%m').dt.quarter

# Create a one-hot encoding for the quarters
quarter_dummies = pd.get_dummies(df['Quarter_Num'], prefix='Q')

# Fit a linear regression model
regression_model = LinearRegression()
regression_model.fit(quarter_dummies, df['Value'])

# Extract the coefficients from the regression model
coefficients = regression_model.coef_

# Calculate the seasonality index
seasonality_index = coefficients / np.mean(coefficients)

print(seasonality_index)


In [ ]:
"""
The Holt-Winters exponential smoothing method is specifically designed to capture seasonality in time series data. 
It models and estimates the seasonal component, allowing for a robust estimation of the seasonality index at the quarter level.
"""

import pandas as pd
from statsmodels.tsa.holtwinters import ExponentialSmoothing

# Create a DataFrame with quarterly data
data = {
    'Quarter': ['Q1', 'Q2', 'Q3', 'Q4', 'Q1', 'Q2', 'Q3', 'Q4'],
    'Value': [100, 110, 120, 130, 140, 150, 160, 170]
}

df = pd.DataFrame(data)

# Convert the quarters to datetime format
df['Quarter'] = pd.to_datetime(df['Quarter'], format='Q%m')

# Convert the DataFrame to a time series
df.set_index('Quarter', inplace=True)

# Apply Holt-Winters exponential smoothing with additive seasonality
model = ExponentialSmoothing(df['Value'], seasonal='add', seasonal_periods=4)
results = model.fit()

# Extract the seasonal component from the fitted model
seasonal_component = results.season

# Calculate the average seasonal component for each quarter
quarterly_avg_seasonality = seasonal_component.groupby(seasonal_component.index.quarter).mean()

# Calculate the overall average
overall_avg = df['Value'].mean()

# Calculate the seasonality index
seasonality_index = quarterly_avg_seasonality / overall_avg

print(seasonality_index)


In [ ]:
print(1)

In [ ]:
#convert a month level dataframe to quarter level
import pandas as pd
import numpy as np

# Create a DataFrame with monthly data
data = {
    'Month': ['2021-01-31', '2021-02-28', '2021-03-31', '2021-04-30', '2021-05-31', '2021-06-30', '2021-07-31', '2021-08-31', '2021-09-30', '2021-10-31', '2021-11-30', '2021-12-31'],
    'Value': [100, 110, 120, 130, 140, 150, 160, 170, 180, 190, 200, 210]
}

df = pd.DataFrame(data)

# Convert the months to datetime objects
df['Month'] = pd.to_datetime(df['Month'], format='%Y-%m-%d')

# Convert the months to numeric values
df['Month_Num'] = df['Month'].dt.month

# Convert the months to quarters
df['Quarter'] = df['Month_Num'].apply(lambda x: np.ceil(x / 3).astype(int))

# Convert the quarters to datetime objects
df['Quarter'] = pd.to_datetime(df['Quarter'], format='%m').dt.to_period('Q')

# Calculate the average value for each quarter
quarterly_avg = df.groupby('Quarter')['Value'].mean()

print(quarterly_avg)


In [26]:
import numpy as np
import pandas as pd
from statsmodels.tsa.seasonal import seasonal_decompose

# Create a time series
df = pd.read_csv("./demo.csv")
df.index = pd.to_datetime(df["Date"],format="%d-%m-%Y")

result = seasonal_decompose(df["Val"], model="multiplicative",period=4)

# Calculate the seasonality index for each quarter
seasonality_index = result.seasonal / result.seasonal.mean()

# Print the seasonality index
print(seasonality_index)


Date
2018-01-01    1.030504
2018-04-01    0.997277
2018-07-01    0.985499
2018-10-01    0.986720
2019-01-01    1.030504
2019-04-01    0.997277
2019-07-01    0.985499
2019-10-01    0.986720
2020-01-01    1.030504
2020-04-01    0.997277
2020-07-01    0.985499
2020-10-01    0.986720
2021-01-01    1.030504
2021-04-01    0.997277
2021-07-01    0.985499
2021-10-01    0.986720
2022-01-01    1.030504
2022-04-01    0.997277
2022-07-01    0.985499
2022-10-01    0.986720
Name: seasonal, dtype: float64


Date
1    1.030504
2    0.997277
3    0.985499
4    0.986720
Name: seasonal, dtype: float64

In [30]:
result.seasonal.mean()

1.0